# Data prep and exploration

* import tables
* explore data
* join relevant info


## To be continued in data prep 2
* data prep
    * dummies
    * feature selection
* export train and test

In [1]:
import pandas as pd
import glob


In [2]:

path = r'data/' # use your path
files_list = glob.glob(path + "/*.csv")
files_list

['data/order.csv',
 'data/disp.csv',
 'data/account.csv',
 'data/card.csv',
 'data/client.csv',
 'data/loan.csv',
 'data/trans.csv',
 'data/sample.csv',
 'data/district.csv']

In [3]:
order = pd.read_csv(files_list[0], index_col=0)
disp = pd.read_csv(files_list[1], index_col=0)
account = pd.read_csv(files_list[2], index_col=0)
card = pd.read_csv(files_list[3], index_col=0)
client = pd.read_csv(files_list[4], index_col=0)
loan = pd.read_csv(files_list[5], index_col=0)
trans = pd.read_csv(files_list[6], index_col=0)
district = pd.read_csv(files_list[8], index_col=0)

/home/learner/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/learner/.local/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
tables_list = [order, disp, account, card, client, loan, trans, district]

In [5]:
[i.shape for i in tables_list]

[(6471, 6),
 (5369, 4),
 (4500, 4),
 (892, 4),
 (5369, 4),
 (682, 7),
 (1056320, 10),
 (77, 16)]

In [6]:
[i.isna().sum() for i in tables_list]

[order_id         0
 account_id       0
 bank_to          0
 account_to       0
 amount           0
 k_symbol      1379
 dtype: int64,
 disp_id       0
 client_id     0
 account_id    0
 type          0
 dtype: int64,
 account_id     0
 district_id    0
 frequency      0
 date           0
 dtype: int64,
 card_id    0
 disp_id    0
 type       0
 issued     0
 dtype: int64,
 client_id      0
 gender         0
 birth_date     0
 district_id    0
 dtype: int64,
 loan_id       0
 account_id    0
 date          0
 amount        0
 duration      0
 payments      0
 status        0
 dtype: int64,
 trans_id           0
 account_id         0
 date               0
 type               0
 operation     183114
 amount             0
 balance            0
 k_symbol      481881
 bank          782812
 account       760931
 dtype: int64,
 district_id    0
 A2             0
 A3             0
 A4             0
 A5             0
 A6             0
 A7             0
 A8             0
 A9             0
 A10  

## Join Tables

Steps and changes for joining tables.

Note: variable creation and feature selection do not occur in this step

#### order
* join to account by account_id
* add 
    * number of orders per account (n_orders)
    * total amount debited (total_debit)
* drop
    * order_id, bank_to, account_to, amount, K_symbol

#### account
* base table
* rename
    * account_ prefix to date and frequency
* drop
    * district_id

#### card
* join to disp by disp_id
* rename
    * card_ prefix to type and issued
* drop
    * card_id
    
#### district
* join to client by district_id
    * assuming most clients bank in the same district as bank
    * however prioritizing client address over bank address 

#### client
* join to disp by client_id

#### disp
* join to account by account_id
* add
    * number of clients for each account
* drop
    * client_id where type is not owner
    * type, disp_id

#### loan
* join to account by account_id
* rename
    * loan_ prefix to date, amount, durayion, payments, status
* drop
    * loan_id

#### trans
* join to account by account_id
* add
    * number of transactions per account (n_trans)
    * total amount transacted (total_trans)
  

In [7]:
#order

order.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,NaN


In [8]:
order.shape

(6471, 6)

In [9]:
order.groupby(['account_id'])['amount'].sum()

account_id
1         2452.0
2        10638.7
3         5001.0
4         3363.0
5         2668.0
          ...   
11328     4674.0
11333     3286.7
11349    17884.0
11359     9274.3
11362    10687.0
Name: amount, Length: 3758, dtype: float64

In [10]:
order2 = pd.DataFrame(order['account_id'].value_counts())

In [11]:
order2 = order2.rename(columns={'account_id':'n_orders'})

order2['account_id'] = order2.index


In [12]:
order2 = order2.join( order.groupby(['account_id'])['amount'].sum(), on='account_id')

In [13]:
order = order2.rename(columns={'amount':'total_debit'}).sort_values(by=['account_id'])


In [14]:
# account

account.head()

,account_id,district_id,frequency,date
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30


In [15]:
account = account.rename(columns={'frequency': 'account_frequency', 'date': 'account_date'})
account = account.replace(['POPLATEK MESICNE', 'POPLATEK TYDNE', 'POPLATEK PO OBRATU'], 
                          ['monthly', 'weekly', 'transaction'])


In [16]:
account = account.drop(columns = 'district_id')
account.head()

,account_id,account_frequency,account_date
0,1,monthly,1995-03-24
1,2,monthly,1993-02-26
2,3,monthly,1997-07-07
3,4,monthly,1996-02-21
4,5,monthly,1997-05-30


In [17]:
# card

card.head()

,card_id,disp_id,type,issued
0,1,9,gold,1998-10-16
1,2,19,classic,1998-03-13
2,3,41,gold,1995-09-03
3,4,42,classic,1998-11-26
4,5,51,junior,1995-04-24


In [18]:
card.drop(columns = 'card_id', inplace=True)
card = card.add_prefix('card_')


In [19]:
card = card.rename(columns = {'card_disp_id': 'disp_id'})

card.head()

,disp_id,card_type,card_issued
0,9,gold,1998-10-16
1,19,classic,1998-03-13
2,41,gold,1995-09-03
3,42,classic,1998-11-26
4,51,junior,1995-04-24


In [20]:
# district

district.head()

,district_id,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,Hl.m. Praha,Prague,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,2,Benesov,central Bohemia,88884,80,26,6,2,5,46.7,8507,1.6,1.85,132,2159.0,2674
2,3,Beroun,central Bohemia,75232,55,26,4,1,5,41.7,8980,1.9,2.21,111,2824.0,2813
3,4,Kladno,central Bohemia,149893,63,29,6,2,6,67.4,9753,4.6,5.05,109,5244.0,5892
4,5,Kolin,central Bohemia,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040


In [21]:
# disp + necessary joins

disp.head()

,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT
3,4,4,3,OWNER
4,5,5,3,DISPONENT


In [22]:
disp = disp.join(card.set_index('disp_id'), on='disp_id')

In [23]:
client = client.join(district.set_index('district_id'), on='district_id')

In [24]:
disp = disp.join(client.set_index('client_id'), on='client_id')

In [25]:
disp.shape, disp.isna().sum()

((5369, 24),
 disp_id           0
 client_id         0
 account_id        0
 type              0
 card_type      4477
 card_issued    4477
 gender            0
 birth_date        0
 district_id       0
 A2                0
 A3                0
 A4                0
 A5                0
 A6                0
 A7                0
 A8                0
 A9                0
 A10               0
 A11               0
 A12              61
 A13               0
 A14               0
 A15              61
 A16               0
 dtype: int64)

In [26]:
n_clients = pd.DataFrame(disp['account_id'].value_counts())
n_clients = n_clients.rename(columns={'account_id':'n_clients'})
n_clients.sample(5)

,n_clients
3287,1
5250,1
299,1
588,1
3148,1


In [27]:
n_clients['account_id'] = n_clients.index

In [28]:
disp = disp.join(n_clients.set_index('account_id'), on='account_id')

In [29]:
disp = disp[disp['type'] != 'DISPONENT']
disp.drop(columns = ['disp_id', 'client_id', 'type'], inplace = True)
disp.head()

,account_id,card_type,card_issued,gender,birth_date,district_id,A2,A3,A4,A5,...,A8,A9,A10,A11,A12,A13,A14,A15,A16,n_clients
0,1,NaN,NaN,F,1970-12-13,18,Pisek,south Bohemia,70699,60,...,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,1
1,2,NaN,NaN,M,1945-02-04,1,Hl.m. Praha,Prague,1204953,0,...,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,2
3,3,NaN,NaN,M,1956-12-01,5,Kolin,central Bohemia,95616,65,...,1,6,51.4,9307,3.8,4.43,118,2616.0,3040,2
5,4,NaN,NaN,M,1919-09-22,12,Pribram,central Bohemia,107870,84,...,1,6,58.0,8754,3.8,4.31,137,3804.0,3868,1
6,5,NaN,NaN,M,1929-01-25,15,Cesky Krumlov,south Bohemia,58796,22,...,1,5,51.9,9045,3.1,3.60,124,1845.0,1879,1


In [30]:
#loan

loan.head()

,loan_id,account_id,date,amount,duration,payments,status
0,4959,2,1994-01-05,80952,24,3373.0,A
1,4961,19,1996-04-29,30276,12,2523.0,B
2,4962,25,1997-12-08,30276,12,2523.0,A
3,4967,37,1998-10-14,318480,60,5308.0,D
4,4968,38,1998-04-19,110736,48,2307.0,C


In [31]:
loan = loan.add_prefix('loan_')
loan = loan.rename(columns={'loan_account_id': 'account_id'})


In [32]:
loan.drop(columns = 'loan_loan_id', inplace=True)
loan.head()

,account_id,loan_date,loan_amount,loan_duration,loan_payments,loan_status
0,2,1994-01-05,80952,24,3373.0,A
1,19,1996-04-29,30276,12,2523.0,B
2,25,1997-12-08,30276,12,2523.0,A
3,37,1998-10-14,318480,60,5308.0,D
4,38,1998-04-19,110736,48,2307.0,C


In [33]:
# trans
trans.head()


,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1,1,1995-03-24,PRIJEM,VKLAD,1000,1000,NaN,NaN,NaN
1,5,1,1995-04-13,PRIJEM,PREVOD Z UCTU,3679,4679,NaN,AB,41403269.0
2,6,1,1995-05-13,PRIJEM,PREVOD Z UCTU,3679,20977,NaN,AB,41403269.0
3,7,1,1995-06-13,PRIJEM,PREVOD Z UCTU,3679,26835,NaN,AB,41403269.0
4,8,1,1995-07-13,PRIJEM,PREVOD Z UCTU,3679,30415,NaN,AB,41403269.0


In [34]:
n_trans = pd.DataFrame(trans['account_id'].value_counts())

n_trans = n_trans.rename(index = str, columns = {'account_id': 'n_trans'})


n_trans['account_id'] = n_trans.index

In [35]:
n_trans.head()

,n_trans,account_id
8261,675,8261
3834,665,3834
96,661,96
2932,655,2932
9307,649,9307


In [36]:
n_trans['account_id'] = n_trans['account_id'].astype('int64')
n_trans = n_trans.join(trans.groupby(['account_id'])['amount'].sum(), on='account_id')
n_trans.head()

,n_trans,account_id,amount
8261,675,8261,3241957
3834,665,3834,2216845
96,661,96,6749870
2932,655,2932,5608802
9307,649,9307,4964583


In [37]:
n_trans = n_trans.rename(columns = {'amount': 'trans_amount'})

In [38]:
n_trans = n_trans.join(trans.sort_values('date').groupby('account_id').tail(1)[['account_id','balance']].set_index('account_id'),
                      on='account_id')

In [39]:
n_trans = n_trans.rename(columns = {'balance': 'latest_balance'})

In [40]:
n_trans.head()

,n_trans,account_id,trans_amount,latest_balance
8261,675,8261,3241957,62967
3834,665,3834,2216845,30658
96,661,96,6749870,84861
2932,655,2932,5608802,33975
9307,649,9307,4964583,111645


In [41]:
trans = n_trans

In [42]:
# join all

disp.head()

,account_id,card_type,card_issued,gender,birth_date,district_id,A2,A3,A4,A5,...,A8,A9,A10,A11,A12,A13,A14,A15,A16,n_clients
0,1,NaN,NaN,F,1970-12-13,18,Pisek,south Bohemia,70699,60,...,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,1
1,2,NaN,NaN,M,1945-02-04,1,Hl.m. Praha,Prague,1204953,0,...,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,2
3,3,NaN,NaN,M,1956-12-01,5,Kolin,central Bohemia,95616,65,...,1,6,51.4,9307,3.8,4.43,118,2616.0,3040,2
5,4,NaN,NaN,M,1919-09-22,12,Pribram,central Bohemia,107870,84,...,1,6,58.0,8754,3.8,4.31,137,3804.0,3868,1
6,5,NaN,NaN,M,1929-01-25,15,Cesky Krumlov,south Bohemia,58796,22,...,1,5,51.9,9045,3.1,3.60,124,1845.0,1879,1


In [43]:
for i in [order, trans, disp, loan]:
    account = account.join(i.set_index('account_id'), on='account_id')

In [44]:
account.columns, account.shape

(Index(['account_id', 'account_frequency', 'account_date', 'n_orders',
        'total_debit', 'n_trans', 'trans_amount', 'latest_balance', 'card_type',
        'card_issued', 'gender', 'birth_date', 'district_id', 'A2', 'A3', 'A4',
        'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15',
        'A16', 'n_clients', 'loan_date', 'loan_amount', 'loan_duration',
        'loan_payments', 'loan_status'],
       dtype='object'),
 (4500, 34))

In [46]:
account.to_csv('data/financial.csv')